In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [3]:
# Load environment variables

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

In [7]:
class Website:
    url:str
    title:str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [9]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows: \
    please provide a short summary of this website in markwon. \
    If it incluses news or announcement, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows:     please provide a short summary of this website in markwon.     If it incluses news or announcement, then summarize these too. 

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt

In [16]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows:     please provide a short summary of this website in markwon.     If it incluses news or announcement, then summarize these too. \n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can

In [18]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages_for(website))
    return response.choices[0].message.content

In [19]:
summarize("https://www.edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website showcases his interests and professional endeavors primarily in coding, LLMs (Large Language Models), and AI. He is the co-founder and CTO of Nebula.io, an AI company focused on talent discovery and engagement. Previously, he founded an AI startup, untapt, which was acquired in 2021. In addition to his work in tech, Ed enjoys DJing and electronic music production.\n\n## Recent News and Announcements\n\n- **September 15, 2025**: Discussion on **AI in Production**: Gen AI and Agentic AI on AWS at scale.\n- **May 28, 2025**: Announcement of a course offering to **become an LLM expert and leader**.\n- **May 18, 2025**: Information about the **2025 AI Executive Briefing**.\n- **April 21, 2025**: Launch of **The Complete Agentic AI Engineering Course**.\n\nThe site encourages visitors to connect with Ed, suggesting a welcoming environment for discussions around AI and talent management."

In [20]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://www.edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal and professional platform where he shares his interests and expertise in programming and large language models (LLMs). Ed is a co-founder and CTO of Nebula.io, which focuses on utilizing AI to help individuals realize their potential and streamline talent management for recruiters. He previously founded the AI startup untapt, which was acquired in 2021. 

## Key Sections:
- **About Ed**: An introduction to Ed's interests in coding, LLMs, DJing, and electronic music. 
- **Professional Background**: Highlights his role at Nebula.io and previous endeavors in the AI industry.
- **News and Announcements**:
  - **Upcoming Events and Courses**:
    - **September 15, 2025** - "AI in Production: Gen AI and Agentic AI on AWS at scale"
    - **May 28, 2025** - "Connecting my courses – become an LLM expert and leader"
    - **May 18, 2025** - "2025 AI Executive Briefing"
    - **April 21, 2025** - "The Complete Agentic AI Engineering Course"

The website encourages visitors to connect with Ed for insights and collaboration opportunities related to AI and technology.